In [ ]:
nvcc --version

In [4]:
# !pip install transformers
!pip install torch
!pip show torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: torch
Version: 2.0.0+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton
<module 'torch.version' from '/usr/local/lib/python3.10/dist-packages/torch/version.py'>


In [ ]:
print(torch.version.cuda)
print(torchvision.cuda_version)

In [2]:
!pip install -qU transformers torch datasets gdcm pydicom pinecone-client

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [3]:
import os
import torch
import skimage
import requests
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import IPython.display
import matplotlib.pyplot as plt
from datasets import load_dataset
from collections import OrderedDict
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

In [4]:
# https://huggingface.co/datasets/conceptual_captions
image_data = load_dataset(
    "conceptual_captions", split="train",
    )

In [5]:
image_data

Dataset({
    features: ['image_url', 'caption'],
    num_rows: 3318333
})

In [6]:
image_data_df = pd.DataFrame(image_data[:100])

In [7]:
image_data_df.shape

(100, 2)

In [8]:
image_data_df.head()

,image_url,caption
0,http://lh6.ggpht.com/-IvRtNLNcG8o/TpFyrudaT6I/...,a very typical bus station
1,http://78.media.tumblr.com/3b133294bdc7c7784b7...,sierra looked stunning in this top and this sk...
2,https://media.gettyimages.com/photos/young-con...,young confused girl standing in front of a war...
3,https://thumb1.shutterstock.com/display_pic_wi...,interior design of modern living room with fir...
4,https://thumb1.shutterstock.com/display_pic_wi...,cybernetic scene isolated on white background .


In [9]:
image_data[0]['image_url']

'http://lh6.ggpht.com/-IvRtNLNcG8o/TpFyrudaT6I/AAAAAAAAM6o/_11MuAAKalQ/IMG_3422.JPG?imgmax=800'

In [29]:
!pip install torch torchvision
# 

# !pip uninstall -y torchvision
# import torch
# print(torch.__version__)

# # !pip install torchvision==2.8 -f https://download.pytorch.org/whl/cu111/torch_stable.html


 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


NameError: ignored

In [30]:
# import torch
# import torchvision
print(torch.version.cuda)
print(torchvision.cuda_version)

11.7


NameError: ignored

In [11]:
!pip install clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [13]:
import torch
import clip
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, image_paths, texts):
        self.image_paths = image_paths
        self.texts = texts
        self.transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        text = self.texts[idx]
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)
        return image, text

# Define your dataset paths and texts
# image_paths = ["path/to/image1.jpg", "path/to/image2.jpg", ...]
# texts = ["description1", "description2", ...]

image_paths = ["path/to/image1.jpg", "path/to/image2.jpg", ...]
texts = ["description1", "description2", ...]

# Initialize and load the pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Create an instance of your custom dataset
dataset = CustomDataset(image_paths, texts)

# Define training parameters
batch_size = 32
num_epochs = 10
learning_rate = 1e-5

# Create a data loader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Set up optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CosineEmbeddingLoss()


RuntimeError: ignored

In [ ]:

# Fine-tune the CLIP model
model.train()
for epoch in range(num_epochs):
    for images, texts in data_loader:
        optimizer.zero_grad()

        # Encode images and texts
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
        image_features = model.encode_image(images)
        text_features = model.encode_text(texts)

        # Compute the loss
        loss = criterion(image_features, text_features, torch.ones(images.shape[0]).to(device))

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Save the fine-tuned model
torch.save(model.state_dict(), "path/to/fine_tuned_model.pt")


In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# import requests

# from transformers import CLIPProcessor, CLIPModel, AdamW

# # Custom dataset class
# class CustomDataset(Dataset):
#     def __init__(self, image_paths, texts, labels):
#         self.image_paths = image_paths
#         self.texts = texts
#         self.labels = labels
#         self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         text = self.texts[idx]
#         label = self.labels[idx]

#         # Load and preprocess image
#         image = Image.open(requests.get(image_path, stream=True).raw)
#         inputs = self.processor(text=[text], images=image, return_tensors="pt", padding=True)
#         input_ids = inputs.input_ids.squeeze()
#         attention_mask = inputs.attention_mask.squeeze()

#         return input_ids, attention_mask, label

# # Custom fine-tuning model head
# class CustomModelHead(torch.nn.Module):
#     def __init__(self, input_dim, num_labels):
#         super(CustomModelHead, self).__init__()
#         self.fc = torch.nn.Linear(input_dim, num_labels)

#     def forward(self, inputs):
#         x = self.fc(inputs)
#         return x




In [ ]:
# # Prepare the dataset

# image = np.asarray(bytearray(requests.get(url).content), dtype="uint8")
# image = cv2.imdecode(image, cv2.IMREAD_COLOR)
# image = Image.fromarray(image)


# image_paths = image_data_df["image_url"]
# texts = image_data_df["caption"]
# labels = [i for i in range(len(texts))]  # Custom labels for each image-text pair

# dataset = CustomDataset(image_paths, texts, labels)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Load the pre-trained CLIP model
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# # Create the custom model head
# # model_head = CustomModelHead(input_dim=model.text_projection.shape[-1], num_labels=2)  # Modify num_labels according to your task
# model_head = CustomModelHead(input_dim=model.text_projection.out_features, num_labels=2)  # Modify num_labels according to your task


# # Define the optimizer and loss function
# optimizer = AdamW(model_head.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# # Fine-tuning loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_head.to(device)
# model.to(device)
# model.train()

# for epoch in range(5):  # Modify the number of epochs as needed
#     total_loss = 0
#     for input_ids, attention_mask, labels in dataloader:
#         input_ids = input_ids.to(device)
#         attention_mask = attention_mask.to(device)
#         labels = labels.to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         text_features = outputs.text_features
#         logits = model_head(text_features)

#         loss = loss_fn(logits, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}: Loss = {total_loss}")


UnidentifiedImageError: ignored

In [ ]:
# # Testing the fine-tuned model
# model.eval()

# # Prepare a test image and text
# test_image_path = "http://path_to_test_image.jpg"
# test_text = "test_text_description"

# test_image = Image.open(requests.get(test_image_path, stream=True).raw)
# test_inputs = dataset.processor(text=[test_text], images=test_image, return_tensors="pt", padding=True)

# test_input_ids = test_inputs.input_ids

In [ ]:
# test_attention_mask = test_inputs.attention_mask

# test_input_ids = test_input_ids.to(device)
# test_attention_mask = test_attention_mask.to(device)

# with torch.no_grad():
#     test_outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask)
#     test_text_features = test_outputs.text_features
#     test_logits = model_head(test_text_features)

#     probabilities = torch.softmax(test_logits, dim=1)
#     predicted_label = torch.argmax(probabilities, dim=1)

# print("Predicted label:", predicted_label.item())
# print("Probabilities:", probabilities.squeeze().tolist())
